In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta 
import nba_api
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
from nba_api.stats.endpoints import playergamelogs


In [2]:
#! pip uninstall nba_api

In [58]:
#NEXT STEP:
#GET THE DATASET FOR THE ENTIRE NBA SEASON

In [2]:
#Get a specific date in the format for PlayerGameLogs endpoint
#delta: number of days to subtract from the current date
def get_date(delta):
    date = datetime.now() - timedelta(days=delta)
    date = date.strftime('%Y-%m-%d') + 'T00:00:00'

    return date

In [3]:
from nba_api.stats.endpoints import playergamelogs
#Get player data from specific date
#date: date for player data
def player_game_logs_date(date):
    #Player Game Log stats for the 2020-21 NBA season
    player_log = playergamelogs.PlayerGameLogs(season_nullable = '2020-21').get_data_frames()[0]
    #.loc[ :, 'PLAYER_NAME': 'AST']
    
    #Grab all game log data for specific player ordered by date
    player_date_log = player_log.loc[player_log['GAME_DATE'] == date].sort_values(by=['GAME_ID'])
    
    return player_date_log[[
            "PLAYER_ID",
            "PLAYER_NAME",
            "TEAM_ID",
            "TEAM_ABBREVIATION",
            "GAME_ID",
            "MATCHUP",
            "WL",
            "MIN",
            "FGM",
            "FGA",
            "FG_PCT",
            "FG3M",
            "FG3A",
            "FG3_PCT",
            "FTM",
            "FTA",
            "FT_PCT",
            "OREB",
            "DREB",
            "REB",
            "AST",
            "PTS"]]

In [15]:
#Get Player game data for a specific date
date = get_date(10)
print(date)
player_date_log = player_game_logs_date(date)

2021-02-28T00:00:00


In [16]:
player_date_log

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,MATCHUP,WL,MIN,FGM,FGA,...,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,PTS
712,201586,Serge Ibaka,1610612746,LAC,0022000520,LAC @ MIL,L,26.746667,6,13,...,7,0.429,0,0,0.000,2,9,11,0,15
788,1626192,Pat Connaughton,1610612749,MIL,0022000520,MIL vs. LAC,W,27.215000,2,5,...,3,0.333,2,2,1.000,1,7,8,0,7
702,201950,Jrue Holiday,1610612749,MIL,0022000520,MIL vs. LAC,W,17.913333,1,5,...,3,0.000,0,0,0.000,0,0,0,3,2
709,203648,Thanasis Antetokounmpo,1610612749,MIL,0022000520,MIL vs. LAC,W,4.935000,0,1,...,1,0.000,0,0,0.000,1,0,1,0,0
646,101150,Lou Williams,1610612746,LAC,0022000520,LAC @ MIL,L,24.783333,5,11,...,3,0.333,3,3,1.000,0,2,2,4,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,1629610,DaQuan Jeffries,1610612758,SAC,0022000528,SAC vs. CHA,L,12.448333,0,2,...,2,0.000,0,0,0.000,1,1,2,0,0
707,202709,Cory Joseph,1610612758,SAC,0022000528,SAC vs. CHA,L,16.118333,0,3,...,2,0.000,0,0,0.000,2,0,2,2,0
706,1628368,De'Aaron Fox,1610612758,SAC,0022000528,SAC vs. CHA,L,40.558333,9,22,...,3,0.333,1,4,0.250,0,1,1,14,20
644,1627741,Buddy Hield,1610612758,SAC,0022000528,SAC vs. CHA,L,38.810000,10,17,...,15,0.533,2,3,0.667,0,2,2,7,30


In [23]:
player_date_log.to_csv(r"C:\Users\Davis Goren\Desktop\nba_test.csv")

In [17]:
game_list = player_date_log['GAME_ID'].unique()

In [18]:
#NEXT STEPS: COMBINE PLAYER AND OPPOSING TEAM STATS
#Advanced stats per game for players and teams
#Players who didn't play in the game are dropped
#game_id_list: List of game id's for a specific duration
def game_advanced_stats(game_id_list):
    advanced_day_df = pd.DataFrame()
    for game_id in game_id_list:
        advanced_score = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id).get_data_frames()
        player_advanced_stats = advanced_score[0][['GAME_ID', 
                                                   'TEAM_ID',
                                                   'TEAM_ABBREVIATION',
                                                   'PLAYER_ID',
                                                   'PLAYER_NAME', 
                                                   'MIN',
                                                   'E_OFF_RATING',
                                                   'OFF_RATING', 
                                                   'TM_TOV_PCT',
                                                   'EFG_PCT',
                                                   'TS_PCT',
                                                   'USG_PCT',
                                                   'E_USG_PCT',
                                                   'E_PACE']].dropna(0)
        #Appending a games advanced stats to the entire days
        advanced_day_df = advanced_day_df.append(player_advanced_stats, ignore_index=True)
        
        team_defensive_rating = advanced_score[1][["GAME_ID",
                                                   "TEAM_ID",
                                                   "TEAM_NAME",
                                                   "E_OFF_RATING",
                                                    "OFF_RATING",
                                                    "E_DEF_RATING",
                                                    "DEF_RATING",
                                                    "E_NET_RATING",
                                                    "NET_RATING"]]

    return advanced_day_df, team_defensive_rating

In [19]:
player_stats, team_stats = game_advanced_stats(game_list)

In [20]:
#Merges the player game data as well as the advanced player data
def merge_player_data(df1, df2):
    player_data_merged = pd.merge(df1, df2, on= ['PLAYER_ID', 'GAME_ID'],
                                suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
    
    return player_data_merged

In [21]:
merged_df = merge_player_data(player_date_log, player_stats)

In [44]:
player_date_log

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,MATCHUP,WL,MIN,FGM,FGA,...,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,PTS
712,201586,Serge Ibaka,1610612746,LAC,0022000520,LAC @ MIL,L,26.746667,6,13,...,7,0.429,0,0,0.000,2,9,11,0,15
788,1626192,Pat Connaughton,1610612749,MIL,0022000520,MIL vs. LAC,W,27.215000,2,5,...,3,0.333,2,2,1.000,1,7,8,0,7
702,201950,Jrue Holiday,1610612749,MIL,0022000520,MIL vs. LAC,W,17.913333,1,5,...,3,0.000,0,0,0.000,0,0,0,3,2
709,203648,Thanasis Antetokounmpo,1610612749,MIL,0022000520,MIL vs. LAC,W,4.935000,0,1,...,1,0.000,0,0,0.000,1,0,1,0,0
646,101150,Lou Williams,1610612746,LAC,0022000520,LAC @ MIL,L,24.783333,5,11,...,3,0.333,3,3,1.000,0,2,2,4,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,1629610,DaQuan Jeffries,1610612758,SAC,0022000528,SAC vs. CHA,L,12.448333,0,2,...,2,0.000,0,0,0.000,1,1,2,0,0
707,202709,Cory Joseph,1610612758,SAC,0022000528,SAC vs. CHA,L,16.118333,0,3,...,2,0.000,0,0,0.000,2,0,2,2,0
706,1628368,De'Aaron Fox,1610612758,SAC,0022000528,SAC vs. CHA,L,40.558333,9,22,...,3,0.333,1,4,0.250,0,1,1,14,20
644,1627741,Buddy Hield,1610612758,SAC,0022000528,SAC vs. CHA,L,38.810000,10,17,...,15,0.533,2,3,0.667,0,2,2,7,30


In [77]:
x = player_date_log.merge(player_stats.loc[:, ['PLAYER_ID', 'GAME_ID']], how = 'inner', on = ['GAME_ID','PLAYER_ID'])

In [78]:
x

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,MATCHUP,WL,MIN,FGM,FGA,...,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,PTS
0,201586,Serge Ibaka,1610612746,LAC,0022000520,LAC @ MIL,L,26.746667,6,13,...,7,0.429,0,0,0.000,2,9,11,0,15
1,1626192,Pat Connaughton,1610612749,MIL,0022000520,MIL vs. LAC,W,27.215000,2,5,...,3,0.333,2,2,1.000,1,7,8,0,7
2,201950,Jrue Holiday,1610612749,MIL,0022000520,MIL vs. LAC,W,17.913333,1,5,...,3,0.000,0,0,0.000,0,0,0,3,2
3,203648,Thanasis Antetokounmpo,1610612749,MIL,0022000520,MIL vs. LAC,W,4.935000,0,1,...,1,0.000,0,0,0.000,1,0,1,0,0
4,101150,Lou Williams,1610612746,LAC,0022000520,LAC @ MIL,L,24.783333,5,11,...,3,0.333,3,3,1.000,0,2,2,4,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,1629610,DaQuan Jeffries,1610612758,SAC,0022000528,SAC vs. CHA,L,12.448333,0,2,...,2,0.000,0,0,0.000,1,1,2,0,0
170,202709,Cory Joseph,1610612758,SAC,0022000528,SAC vs. CHA,L,16.118333,0,3,...,2,0.000,0,0,0.000,2,0,2,2,0
171,1628368,De'Aaron Fox,1610612758,SAC,0022000528,SAC vs. CHA,L,40.558333,9,22,...,3,0.333,1,4,0.250,0,1,1,14,20
172,1627741,Buddy Hield,1610612758,SAC,0022000528,SAC vs. CHA,L,38.810000,10,17,...,15,0.533,2,3,0.667,0,2,2,7,30


In [71]:
merged_df

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,MATCHUP,WL,MIN,FGM,FGA,...,AST,PTS,E_OFF_RATING,OFF_RATING,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE
0,201586,Serge Ibaka,1610612746,LAC,0022000520,LAC @ MIL,L,26.746667,6,13,...,0,15,109.9,107.5,0.0,0.577,0.577,0.217,0.221,92.67
1,1626192,Pat Connaughton,1610612749,MIL,0022000520,MIL vs. LAC,W,27.215000,2,5,...,0,7,108.0,101.8,0.0,0.500,0.595,0.100,0.106,98.31
2,201950,Jrue Holiday,1610612749,MIL,0022000520,MIL vs. LAC,W,17.913333,1,5,...,3,2,115.9,111.1,20.0,0.200,0.200,0.175,0.187,95.50
3,203648,Thanasis Antetokounmpo,1610612749,MIL,0022000520,MIL vs. LAC,W,4.935000,0,1,...,0,0,141.7,140.0,0.0,0.000,0.000,0.091,0.092,93.37
4,101150,Lou Williams,1610612746,LAC,0022000520,LAC @ MIL,L,24.783333,5,11,...,4,14,111.4,107.4,11.1,0.500,0.568,0.226,0.237,97.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,1629610,DaQuan Jeffries,1610612758,SAC,0022000528,SAC vs. CHA,L,12.448333,0,2,...,0,0,106.0,109.1,0.0,0.000,0.000,0.080,0.081,89.23
170,202709,Cory Joseph,1610612758,SAC,0022000528,SAC vs. CHA,L,16.118333,0,3,...,2,0,97.5,93.8,28.6,0.000,0.000,0.135,0.144,97.20
171,1628368,De'Aaron Fox,1610612758,SAC,0022000528,SAC vs. CHA,L,40.558333,9,22,...,14,20,127.1,125.9,5.1,0.432,0.421,0.269,0.277,101.68
172,1627741,Buddy Hield,1610612758,SAC,0022000528,SAC vs. CHA,L,38.810000,10,17,...,7,30,130.7,131.8,3.8,0.824,0.819,0.207,0.207,106.19
